# Day 8: Sector Rotation Strategies

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week2_position_trend/day08_sector_rotation.ipynb)

---

## Learning Objectives

1. **Understand** sector rotation and business cycles
2. **Identify** leading and lagging sectors
3. **Apply** relative strength analysis to sectors
4. **Build** a sector rotation trading system
5. **Time** entries based on sector momentum

---

## Lecture (30 minutes)

### What is Sector Rotation?

**Sector rotation** is the strategy of moving money between industry sectors based on the business cycle and relative performance.

```
SECTOR ROTATION CONCEPT
=======================

Different sectors outperform at different stages:

EARLY EXPANSION:     LATE EXPANSION:
- Technology         - Energy
- Consumer Disc.     - Materials
- Financials         - Industrials

EARLY CONTRACTION:   LATE CONTRACTION:
- Utilities          - Consumer Staples
- Healthcare         - Healthcare
- Consumer Staples   - Utilities
```

### The Business Cycle

```
BUSINESS CYCLE PHASES
=====================

         EXPANSION              CONTRACTION
    [Early]  [Late]         [Early]   [Late]
       |       |               |         |
       v       v               v         v
    ___/\                          /\___
   /     \____Peak____           /      \___Trough___
  /                   \         /                    \
 /                     \_______/                      \

LEADING SECTORS:
Early Expansion: Tech, Financials, Consumer Discretionary
Late Expansion: Energy, Materials, Industrials
Early Contraction: Utilities, Healthcare
Late Contraction: Consumer Staples
```

### Sector ETFs

```
S&P 500 SECTOR ETFs (SPDR)
==========================

XLF - Financials        XLU - Utilities
XLK - Technology        XLE - Energy
XLV - Healthcare        XLI - Industrials
XLY - Consumer Disc.    XLB - Materials
XLP - Consumer Staples  XLRE - Real Estate
XLC - Communications

CYCLICAL (move with economy):
XLY, XLK, XLF, XLI, XLB, XLE

DEFENSIVE (stable in downturns):
XLU, XLP, XLV, XLRE
```

### Relative Strength Analysis

```
MEASURING SECTOR STRENGTH
=========================

RELATIVE STRENGTH RATIO:
RS = Sector Price / SPY Price

INTERPRETATION:
- Rising RS = Sector outperforming market
- Falling RS = Sector underperforming market
- RS > 1 crossover = Buy signal
- RS < 1 crossover = Sell signal

ROTATION SIGNAL:
Rotate into sectors with:
- Rising relative strength
- Breaking above RS moving average
- New RS highs
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
# Define sector ETFs
sectors = {
    'XLK': 'Technology',
    'XLF': 'Financials',
    'XLV': 'Healthcare',
    'XLY': 'Consumer Disc.',
    'XLP': 'Consumer Staples',
    'XLE': 'Energy',
    'XLU': 'Utilities',
    'XLI': 'Industrials',
    'XLB': 'Materials'
}

def fetch_sector_data(tickers, period='1y'):
    """Fetch data for multiple tickers."""
    data = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            df = stock.history(period=period)
            data[ticker] = df['Close']
        except:
            pass
    return pd.DataFrame(data)

# Fetch sector data
tickers = list(sectors.keys()) + ['SPY']
prices = fetch_sector_data(tickers)
print(f"Loaded data for {len(prices.columns)} tickers")

In [ ]:
def analyze_sector_rotation(prices):
    """Analyze sector relative strength."""
    results = []
    
    spy = prices['SPY']
    
    for ticker in prices.columns:
        if ticker == 'SPY':
            continue
        
        sector_price = prices[ticker]
        
        # Calculate relative strength
        rs = sector_price / spy
        rs_ma = rs.rolling(20).mean()
        
        # Performance metrics
        perf_1m = (sector_price.iloc[-1] / sector_price.iloc[-21] - 1) * 100
        perf_3m = (sector_price.iloc[-1] / sector_price.iloc[-63] - 1) * 100 if len(sector_price) > 63 else 0
        
        # RS trend
        rs_current = rs.iloc[-1]
        rs_ma_current = rs_ma.iloc[-1]
        rs_trend = 'UP' if rs_current > rs_ma_current else 'DOWN'
        
        results.append({
            'Ticker': ticker,
            'Sector': sectors.get(ticker, ticker),
            '1M Perf %': perf_1m,
            '3M Perf %': perf_3m,
            'RS vs SPY': rs_current,
            'RS Trend': rs_trend
        })
    
    return pd.DataFrame(results).sort_values('1M Perf %', ascending=False)

# Analyze sectors
sector_analysis = analyze_sector_rotation(prices)

print(f"\n{'='*70}")
print("SECTOR ROTATION ANALYSIS")
print(f"{'='*70}\n")
print(sector_analysis.round(2).to_string(index=False))

# Recommendations
print(f"\n{'='*70}")
print("ROTATION RECOMMENDATIONS:")
print(f"{'='*70}")
leaders = sector_analysis[sector_analysis['RS Trend'] == 'UP'].head(3)
laggards = sector_analysis[sector_analysis['RS Trend'] == 'DOWN'].tail(3)

print("\nSECTORS TO FAVOR (Strong RS):")
for _, row in leaders.iterrows():
    print(f"  {row['Sector']} ({row['Ticker']}): {row['1M Perf %']:+.1f}%")

print("\nSECTORS TO AVOID (Weak RS):")
for _, row in laggards.iterrows():
    print(f"  {row['Sector']} ({row['Ticker']}): {row['1M Perf %']:+.1f}%")

In [ ]:
def plot_sector_performance(prices, sectors):
    """Plot sector relative performance."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Normalize prices to start at 100
    normalized = (prices / prices.iloc[0]) * 100
    
    # Performance chart
    ax1 = axes[0]
    for ticker in normalized.columns:
        if ticker != 'SPY':
            ax1.plot(normalized[ticker], label=sectors.get(ticker, ticker), alpha=0.7)
    ax1.plot(normalized['SPY'], 'k-', lw=2, label='SPY')
    ax1.set_title('Sector Performance (Normalized)')
    ax1.legend(loc='upper left', fontsize=8)
    ax1.grid(True, alpha=0.3)
    
    # Relative strength chart
    ax2 = axes[1]
    for ticker in prices.columns:
        if ticker != 'SPY':
            rs = prices[ticker] / prices['SPY']
            rs_norm = (rs / rs.iloc[0]) * 100
            ax2.plot(rs_norm, label=sectors.get(ticker, ticker), alpha=0.7)
    ax2.axhline(y=100, color='black', linestyle='--')
    ax2.set_title('Relative Strength vs SPY')
    ax2.legend(loc='upper left', fontsize=8)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_sector_performance(prices, sectors)

---

## Summary

- **Sector rotation** moves capital to strongest sectors
- **Business cycle** influences sector performance
- Use **relative strength** to identify leaders/laggards
- **Rotate into** rising RS, **avoid** falling RS

**Tomorrow**: Day 9 - Earnings Trading Strategies